https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb From this link with different data

https://github.com/mhjabreel/CharCnn_Keras/blob/master/models/char_cnn_zhang.py char CNN

In [22]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Embedding
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Input, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report

In [3]:
import sys  
sys.path.insert(0, '../../PreProcessing/')

from utils import *
from parse_df import *
from split_name import *
from char_utils import *

In [4]:
print(tf.__version__)

2.4.0


### PreProcessing

In [8]:
er_df = getERData("../")
er_df = er_df.rename(columns = {'name':'Name', 'gender':'Gender'})
er_df = cleanDf(er_df, 'Name')

/home/souvic/mounted/btp/vahini/Models/ERTrain/../../PreProcessing/parse_df.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df['Name'].str.lower()
/home/souvic/mounted/btp/vahini/Models/ERTrain/../../PreProcessing/parse_df.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len']= [len(str(i)) for i in df[col]]


Gender
FEMALE    10405236
MALE      11632598
Name: Name, dtype: int64


In [34]:
er_df = er_df.dropna(subset=['Gender'])
er_df = er_df.reset_index(drop=True)

In [35]:
er_df.shape

(22037835, 9)

In [38]:
er_df.groupby('Gender')['Gender'].value_counts()

Gender  Gender
FEMALE  FEMALE    10405236
MALE    MALE      11632599
Name: Gender, dtype: int64

In [11]:
train_x, train_y, test_x, test_y = splitERData(er_df, 'Name', 'char_lstm_er_oov.txt')

Train: ((14776538, 30),(14776538, 2))  Test: ((6332842, 30),(6332842, 2))


In [18]:
voc = vocab()

In [40]:
test_x.shape[0]+train_x.shape[0]

21109380

### Build Model

In [41]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

In [42]:
inputs = Input(MAXLEN_NAME_CHAR_LSTM)
x = Embedding(len(voc)+1, len(voc)+1)(inputs)
x = LSTM(512, return_sequences=True, activation='tanh')(x)
x = Dropout(0.2)(x)
x = LSTM(512, return_sequences=False, activation='tanh')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

In [43]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 29)            841       
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 512)           1110016   
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [45]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [46]:
batch_size=1000

In [47]:
model.fit(train_x, train_y,batch_size=batch_size,epochs=10,validation_data=(test_x, test_y))

Epoch 1/10
 8983/14777 [=================>............] - ETA: 30:43 - loss: 0.3048 - accuracy: 0.8610

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



14777/14777 [==============================] - 5192s 351ms/step - loss: 0.2620 - accuracy: 0.8851 - val_loss: 0.1345 - val_accuracy: 0.9524
Epoch 2/10
14777/14777 [==============================] - 5153s 349ms/step - loss: 0.1306 - accuracy: 0.9544 - val_loss: 0.1252 - val_accuracy: 0.9566
Epoch 3/10
 2601/14777 [====>.........................] - ETA: 1:03:55 - loss: 0.1196 - accuracy: 0.9590

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



13377/14777 [==========================>...] - ETA: 7:21 - loss: 0.1204 - accuracy: 0.9588

...

 6405/14777 [============>.................] - ETA: 1:16:28 - loss: 0.1038 - accuracy: 0.9662

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



14777/14777 [==============================] - 7733s 523ms/step - loss: 0.1044 - accuracy: 0.9658 - val_loss: 0.1189 - val_accuracy: 0.9600
Epoch 10/10
 5088/14777 [=========>....................] - ETA: 50:51 - loss: 0.1015 - accuracy: 0.9669

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



14777/14777 [==============================] - 5142s 348ms/step - loss: 0.1027 - accuracy: 0.9664 - val_loss: 0.1197 - val_accuracy: 0.9599


In [50]:
print("hello")

hello


In [51]:
score, acc = model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

197902/197902 [==============================] - 5228s 26ms/step - loss: 0.1197 - accuracy: 0.9599
Test score: 0.11973106861114502
Test accuracy: 0.9599427580833435


In [56]:
filename = "SavedModels/FinalSet/CharLSTM/ERModel/ERModel"
model.save(filename)

INFO:tensorflow:Assets written to: SavedModels/FinalSet/CharLSTM/ERModel/ERModel/assets


In [53]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male       0.96      0.97      0.96   3299170
      Female       0.96      0.95      0.96   3033672

    accuracy                           0.96   6332842
   macro avg       0.96      0.96      0.96   6332842
weighted avg       0.96      0.96      0.96   6332842

--------------------------------------------------------


In [58]:
from tensorflow.python.keras.layers import Input, Embedding, Activation, Flatten, Dense
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, Dropout
from tensorflow.python.keras.models import Model
from sklearn.metrics import classification_report

CHAR CNN

In [59]:
inputs = Input(MAXLEN_NAME_CHAR_CNN)
x = Embedding(len(voc)+1, len(voc)+1)(inputs)
x = Conv1D(filters = 256, kernel_size = 7, activation = 'relu')(x)
x = Flatten()(x)
predictions = Dense(2, activation='softmax')(x)
cnn_model = Model(inputs=inputs, outputs=predictions)
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [61]:
cnn_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 30, 29)            841       
_________________________________________________________________
conv1d (Conv1D)              (None, 24, 256)           52224     
_________________________________________________________________
flatten (Flatten)            (None, 6144)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 12290     
Total params: 65,355
Trainable params: 65,355
Non-trainable params: 0
_________________________________________________________________


In [62]:
batch_size=1000
cnn_model.fit(train_x, train_y,batch_size=batch_size,epochs=15,validation_data=(test_x, test_y))

Epoch 1/15


2021-10-28 15:27:58.276923: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-10-28 15:28:08.443063: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-10-28 15:28:09.520029: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


14777/14777 [==============================] - 268s 16ms/step - loss: 0.2400 - accuracy: 0.9032 - val_loss: 0.1836 - val_accuracy: 0.9324
Epoch 2/15
14777/14777 [==============================] - 195s 13ms/step - loss: 0.1714 - accuracy: 0.9379 - val_loss: 0.1680 - val_accuracy: 0.9398
Epoch 3/15
14777/14777 [==============================] - 195s 13ms/step - loss: 0.1650 - accuracy: 0.9409 - val_loss: 0.1637 - val_accuracy: 0.9414
Epoch 4/15
14777/14777 [==============================] - 195s 13ms/step - loss: 0.1617 - accuracy: 0.9423 - val_loss: 0.1627 - val_accuracy: 0.9422
Epoch 5/15
14777/14777 [==============================] - 196s 13ms/step - loss: 0.1595 - accuracy: 0.9432 - val_loss: 0.1608 - val_accuracy: 0.9426
Epoch 6/15
14777/14777 [==============================] - 197s 13ms/step - loss: 0.1579 - accuracy: 0.9440 - val_loss: 0.1593 - val_accuracy: 0.9434
Epoch 7/15
14777/14777 [==============================] - 196s 13ms/step - loss: 0.1569 - accuracy: 0.9445 - val_loss

In [64]:
score, acc = cnn_model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

197902/197902 [==============================] - 552s 3ms/step - loss: 0.1547 - accuracy: 0.9456
Test score: 0.15472562611103058
Test accuracy: 0.9456402063369751


In [65]:
pred = cnn_model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male       0.94      0.95      0.95   3299170
      Female       0.95      0.94      0.94   3033672

    accuracy                           0.95   6332842
   macro avg       0.95      0.95      0.95   6332842
weighted avg       0.95      0.95      0.95   6332842

--------------------------------------------------------


In [66]:
filename = "../SavedModels/FinalSet/CharCNN/ERModel/ERModel"
cnn_model.save(filename)

INFO:tensorflow:Assets written to: ../SavedModels/FinalSet/CharCNN/ERModel/ERModel/assets


In [67]:
filename = "SavedModels/FinalSet/CharCNN/ERModel/ERModel"
cnn_model.save(filename)

INFO:tensorflow:Assets written to: SavedModels/FinalSet/CharCNN/ERModel/ERModel/assets
